In [2]:
pip install flask-socketio


  Obtaining dependency information for flask-socketio from https://files.pythonhosted.org/packages/56/c8/dc0be9e26272dc89342868ecc2d9ddb9e31002b4b8e49fdb754aa0f9ecbf/Flask_SocketIO-5.3.6-py3-none-any.whl.metadata
  Obtaining dependency information for python-socketio>=5.0.2 from https://files.pythonhosted.org/packages/f8/bf/4790ed063ca2daa58fb20285fc3707218cf01e174209355d081d83094f6d/python_socketio-5.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for python-engineio>=4.8.0 from https://files.pythonhosted.org/packages/8b/89/70ebee15c1ef37d3a5408dbb03e57aa226dc6f1921735ed8cd59ac2a0136/python_engineio-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for simple-websocket>=0.10.0 from https://files.pythonhosted.org/packages/6d/ea/288a8ac1d9551354488ff60c0ac6a76acc3b6b60f0460ac1944c75e240da/simple_websocket-1.0.0-py3-none-any.whl.metadata
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   --------

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
from flask import Flask, Response
from flask_socketio import SocketIO

app = Flask(__name__)

# Flask-SocketIO에서 CORS를 모든 도메인에서 접근 허용으로 설정합니다.
socketio = SocketIO(app, cors_allowed_origins="*")

# 전역 변수로 웹캠 초기화
cap = cv2.VideoCapture(0)

# 전역 변수로 Mediapipe Holistic 모델 초기화
holistic = mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 전역 변수로 Mediapipe 그리기 도구 초기화
mp_drawing = mp.solutions.drawing_utils

@app.route("/")
def bye():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

def generate_frames():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 랜드마크 데이터 추출
        landmarks = []

        if results.pose_landmarks:
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                landmarks.append([idx, lm.x, lm.y, lm.z])

        # 랜드마크 데이터를 DataFrame으로 변환
        landmarks_data = pd.DataFrame(landmarks, columns=["Index", "X", "Y", "Z"])
        print(landmarks_data)
        
        # CSV 파일로 저장
        
        #file_path = "C:/User/gjaischool1/Desktop/Aischool/Python/flask/landmarks_data.csv"
        #landmarks_data.to_csv(file_path, mode='a', header=False, index=False)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        ret, buffer = cv2.imencode('.jpg', image)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

if __name__ == "__main__":
    app.run(host="192.168.0.98", port=9000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.0.98:9000
Press CTRL+C to quit
192.168.0.98 - - [08/Nov/2023 09:43:04] "GET /socket.io/?EIO=4&transport=polling&t=OkiOVdV HTTP/1.1" 200 -
192.168.0.98 - - [08/Nov/2023 09:43:04] "POST /socket.io/?EIO=4&transport=polling&t=OkiOVep&sid=d3fsUMnsgWybYP_ZAAAA HTTP/1.1" 200 -
192.168.0.98 - - [08/Nov/2023 09:43:04] "GET /socket.io/?EIO=4&transport=polling&t=OkiOVer&sid=d3fsUMnsgWybYP_ZAAAA HTTP/1.1" 200 -
192.168.0.98 - - [08/Nov/2023 09:43:04] "GET /socket.io/?EIO=4&transport=polling&t=OkiOVg3&sid=d3fsUMnsgWybYP_ZAAAA HTTP/1.1" 200 -
192.168.0.98 - - [08/Nov/2023 09:44:22] "GET /socket.io/?EIO=4&transport=websocket&sid=d3fsUMnsgWybYP_ZAAAA HTTP/1.1" 200 -
